<a href="https://colab.research.google.com/github/GawainGan/LLM/blob/main/Module%202/Module_2_3_FLAN_T5XL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Package

In [45]:
# # Install Hugging Face Transformers and Accelerate
# !pip install -U transformers accelerate bitsandbytes

# # install the Hugging Face Hub and ipywidgets for model management and interactive widgets
# !pip install huggingface_hub
# !pip install ipywidgets

In [2]:
!nvidia-smi # make sure that we are using the T4 GPU

Tue Jul  9 01:11:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Import packages

In [3]:
from huggingface_hub import notebook_login
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import Markdown, display

### Hugging Face long in (Use the Granted Access Token)

In [4]:
notebook_login()

### Load Model [Google flan-t5-xl](https://huggingface.co/google/flan-t5-xl)

In [5]:
# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

### Example1: Running the model on a single / multi GPU

In [6]:
input_text = "translate English to German: This is great?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda") # 仅inputs_ids的张量被移动到GPU上
# input_ids = tokenizer(input_text, return_tensors="pt").to("cuda") # 字典中的所有张量（包括 input_ids 和其他可能的张量，如 attention_mask 等）都被移动到 GPU 上

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Das ist wunderschön?</s>


In [7]:
input_text = "translate English to German: This is great?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

<pad> Das ist wunderschön?</s>


In [8]:
tokenizer.decode(outputs[0])

'<pad> Das ist wunderschön?</s>'

### Example 1.2 Chosecasing the difference of `inputs_ids`

In [9]:
# 输入文本
input_text = "translate English to German: This is great?"

# 方法1：只移动 input_ids
inputs = tokenizer(input_text, return_tensors="pt")
input_ids = inputs.input_ids.to("cuda")
print(f"Method 1 - input_ids size: {input_ids.size()}, device: {input_ids.device}")

# 检查其他张量是否移动到 GPU
if hasattr(inputs, "attention_mask"):
    attention_mask = inputs.attention_mask
    print(f"Method 1 - attention_mask size: {attention_mask.size()}, device: {attention_mask.device}")

# 生成输出
outputs1 = model.generate(input_ids=input_ids)
print(tokenizer.decode(outputs1[0]))

# 只将 input_ids 张量移动到 GPU 上，而 attention_mask 张量保持在 CPU 上

Method 1 - input_ids size: torch.Size([1, 10]), device: cuda:0
Method 1 - attention_mask size: torch.Size([1, 10]), device: cpu
<pad> Das ist wunderschön?</s>


In [10]:
# 方法2：移动整个字典
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
print(f"Method 2 - input_ids size: {inputs.input_ids.size()}, device: {inputs.input_ids.device}")

# 检查其他张量是否移动到 GPU
if hasattr(inputs, "attention_mask"):
    attention_mask = inputs.attention_mask
    print(f"Method 2 - attention_mask size: {attention_mask.size()}, device: {attention_mask.device}")

# 生成输出
outputs2 = model.generate(input_ids=inputs.input_ids)
print(tokenizer.decode(outputs2[0]))

# 将整个包含所有张量的字典移动到 GPU 上，因此 input_ids 和 attention_mask 都被移动到 GPU 上

Method 2 - input_ids size: torch.Size([1, 10]), device: cuda:0
Method 2 - attention_mask size: torch.Size([1, 10]), device: cuda:0
<pad> Das ist wunderschön?</s>


在模型推理过程中，模型通常需要 input_ids 和 attention_mask。如果 attention_mask 仍在 CPU 上，可能会导致数据传输的额外开销或潜在的错误。因此，确保所有相关张量都在同一设备上非常重要。

因此，第二种方法可以确保所有相关的张量（如 input_ids 和 attention_mask）都被移动到 GPU 上，避免手动检查和移动每个张量可能带来的错误或遗漏。

### Example2.1: single user interaction with prompt

In [16]:
# 输入文本
input_text = "The Capricornus is better than others because:"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

# 设置生成参数
generate_params = {
    "max_length": 200,        # 设置生成文本的最大长度 (max 300)
    "num_beams": 7,           # 设置束搜索的束数 (5-10)
    "do_sample": True,        # 启用采样方法
    "temperature": 1.5,       # 设置温度，影响生成的多样性 (0.7-1.5)
    "top_k": 50,              # 设置top_k，限制采样池的大小
    "top_p": 0.95             # 设置top_p，使用累积概率来裁剪采样池
}

# 生成输出
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_length=generate_params["max_length"],
    num_beams=generate_params["num_beams"],
    do_sample=generate_params["do_sample"],
    temperature=generate_params["temperature"],
    top_k=generate_params["top_k"],
    top_p=generate_params["top_p"]
)

# 解码输出
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)

it has a lot of nebulae and globules.


# Run LLM in the Web App (gradio)

In [22]:
# pip install gradio

In [23]:
import gradio as gr

In [25]:
# 模型名称
# model_name = "google/flan-t5-small"
# model_name = "google/flan-t5-base"
model_name = "google/flan-t5-large"
# model_name = "google/flan-t5-xl"

default_max_length = 200

print("Using `{}`.".format(model_name))

# 加载预训练的 T5 分词器和模型
tokenizer = T5Tokenizer.from_pretrained(model_name)
print("T5Tokenizer loaded from pretrained.")

model = T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto")
print("T5ForConditionalGeneration loaded from pretrained.")

Using `google/flan-t5-large`.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5Tokenizer loaded from pretrained.
T5ForConditionalGeneration loaded from pretrained.


In [28]:
def inference(max_length, input_text, history=[]):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

    # 设置生成参数
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_beams=5,     # 使用束搜索
        do_sample=True,  # 启用采样
        temperature=1.2, # 设置温度
        top_k=50,        # 设置top_k
        top_p=0.95       # 设置top_p
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    history.append((input_text, result))
    return history, history

In [29]:

with gr.Blocks() as demo:
    with gr.Row():
        gr.Markdown(
            "<h1>Demo of {}</h1><p>See more at Hugging Face: <a href='https://huggingface.co/{}'>{}</a>.</p>".format(
                model_name, model_name, model_name
            )
        )
        max_length = gr.Number(
            value=default_max_length, label="Maximum length of response"
        )

    chatbot = gr.Chatbot(label=model_name)
    state = gr.State([])

    with gr.Row():
        txt = gr.Textbox(
            show_label=False,
            placeholder="Enter text and press enter",
            container=False)

    txt.submit(fn=inference, inputs=[max_length, txt, state], outputs=[chatbot, state])

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3f01084d0a5d9bb9aa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Run LLM with defined function

In [30]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


--2024-07-09 01:40:33--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-09 01:40:34 (50.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [31]:
import requests
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [40]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [41]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [39]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, )
    result = tokenizer.decode(outputs[0])
    return result

In [43]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [44]:
rag("I just discovered the course. Can I still join it?")

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."